In [1]:
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_VISIBLE_DEVICES=2


In [4]:
!python ../scripts/vinbig/precompute_phrase_embeddings.py \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--average_top_k_most_similar \
--top_k 100 \
--facts_relevant_to_anchor_facts_pickle_filepath \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_relevant_to_anchor_facts(af=25,rf=317035)(hash=411,354564789477999375).pkl"

checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
Loading facts_relevant_to_anchor_facts from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_relevant_to_anchor_facts(af=25,rf=317035)(hash=411,354564789477999375).pkl...
Extracting embeddings...
  0%|                                                | 0/317035 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["hashes"]) = 1570508
self.cache["embeddings"].shape = (1570508, 128)
100%|███████████████████████████████████████| 28/28 [00:00<00:00, 204957.26it/s]
Aortic enlargement: len(anchor_fact_idxs)=2305
Atelectasis: len(anchor_fact_idxs)=32094
Calcification: len(anchor_fact_idxs)=11033
Cardiomegaly: len(anchor_fact_idxs)=7010
Clavicle fracture: len(an

In [6]:
from medvqa.utils.files import load_pickle
tmp = load_pickle('/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/vinbig/label_phrase_embeddings(hash=561,2254045888337408829).pkl')

In [7]:
tmp.keys()

dict_keys(['phrases', 'phrase_embeddings', 'most_similar_facts', 'most_similar_fact_embeddings', 'most_similar_fact_similarities'])

In [ ]:
!python ../train_phrase_grounding.py \
--epochs 200 \
--batches_per_epoch 1500 \
--max_images_per_batch 10 \
--max_phrases_per_batch 1000 \
--max_phrases_per_image 45 \
--val_batch_size_factor 4.0 \
--raw_image_encoding "cxrmate-rrg24-uniformer-huggingface" \
--huggingface_model_name "aehrc/cxrmate-rrg24" \
--image_size 384 384 \
--image_local_feat_size 512 \
--num_regions 144 \
--regions_width 12 \
--regions_height 12 \
--img_aug_mode "random-color-and-spatial" \
--phrase_embedding_size 128 \
--phrase_grounding_mode "adaptive_film_based_pooling_mlp_with_bbox_regression" \
--visual_grounding_hidden_size 256 \
--phrase_mlp_hidden_dims 256 128 \
--num_train_workers 2 \
--num_val_workers 2 \
--gradient_accumulation_steps 4 \
--optimizer_name "adamw" \
--scheduler "exp-warmup+decay+cyclicdecay" \
--lr 1e-6 \
--warmup_decay_and_cyclic_decay_args "1e-6,3,8e-5,5,1e-6,8e-5,5,1e-6" \
--binary_multilabel_classif_loss_name "focal+bce+npbbce" \
--do_visual_grounding_with_bbox_regression \
--use_mimiccxr_facts_for_train \
--use_mimiccxr_facts_for_test \
--dicom_id_to_pos_neg_facts_filepath \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_rel_facts=317035,num_clusters=100,max_neg=500,ent_th=0.172)(hash=1200,2067390724509563020).pkl" \
--mimiccxr_exclude_noisy_images \
--use_vinbig_for_train \
--use_vinbig_for_test \
--vinbig_phrase_embeddings_filepath \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/vinbig/label_phrase_embeddings(hash=561,2254045888337408829).pkl" \
--vinbig_weight 0.1 \
--mimiccxr_facts_weight 0.9 \
--save

script's arguments:
   epochs: 200
   batches_per_epoch: 1500
   max_images_per_batch: 10
   max_phrases_per_batch: 1000
   max_phrases_per_image: 45
   val_batch_size_factor: 4.0
   checkpoint_folder: None
   pretrained_checkpoint_folder_path: None
   pretrained_checkpoint_folder_paths: None
   freeze_image_encoder: False
   raw_image_encoding: cxrmate-rrg24-uniformer-huggingface
   huggingface_model_name: aehrc/cxrmate-rrg24
   num_regions: 144
   image_local_feat_size: 512
   image_encoder_pretrained_weights_path: None
   image_encoder_dropout_p: 0
   yolov8_model_name_or_path: None
   yolov8_model_alias: None
   phrase_embedding_size: 128
   regions_width: 12
   regions_height: 12
   qkv_size: None
   phrase_grounding_mode: adaptive_film_based_pooling_mlp_with_bbox_regression
   phrase_classifier_hidden_size: None
   transf_d_model: None
   transf_nhead: None
   transf_dim_feedforward: None
   transf_dropout: 0
   transf_num_layers: None
   visual_feature_proj_size: None
   visual_

self.phrase_grounding_masks.shape = (18000, 22, 144)
self.phrase_classification_labels.shape = (18000, 22)
Append additional labels to phrase_classification_labels
len(non_bbox_labels) = 6
non_bbox_labels = ['COPD', 'Lung tumor', 'Pneumonia', 'Tuberculosis', 'Other disease', 'No finding']
self.phrase_classification_labels.shape = (18000, 28)
Reorder phrases and phrase_embeddings
len(phrases) = 28
len(actual_phrases) = 28
phrase_embeddings.shape = (28, 128)
Generating train dataset and dataloader
len(train_indices) = 15000
No labels: 0
Edema: 13
Clavicle fracture: 27
Lung cyst: 33
COPD: 35
Lung cavity: 51
Emphysema: 81
Rib fracture: 90
Pneumothorax: 96
Enlarged PA: 131
Mediastinal shift: 150
Atelectasis: 187
Lung tumor: 291
Consolidation: 353
ILD: 397
Calcification: 458
Infiltration: 613
Tuberculosis: 750
Nodule/Mass: 841
Pneumonia: 917
Pleural effusion: 1038
Other lesion: 1154
Lung Opacity: 1331
Pulmonary fibrosis: 1621
Pleural thickening: 2010
Cardiomegaly: 2316
Aortic enlargement: 30

New checkpoint saved: checkpoint_5_miss+miuc+vbss+vbou+vbuc+vbss=0.3630.pt
---- Epoch 6/200
(1) Training stage (lr = 0.000014) ...
loss 1.39744, mimfg_phrcls_loss 1.26318, mimfg_prc_auc 0.62679, vbg_vgbbox_loss 0.38008, vbg_att_sup_loss 0.95717, vbg_phrcls_loss 1.26846, vbg_prc_auc 0.34871, 631.55 secs
(2) Validation stage ...
mimfg_phrcls_loss 1.21198, mimfg_prc_auc 0.66792, vbg_bbox_iou 0.11366, vbg_prc_auc 0.29179, 152.73 secs
New checkpoint saved: checkpoint_6_miss+miuc+vbss+vbou+vbuc+vbss=0.3703.pt
---- Epoch 7/200
(1) Training stage (lr = 0.000006) ...
loss 1.36708, mimfg_phrcls_loss 1.23561, mimfg_prc_auc 0.63867, vbg_vgbbox_loss 0.37058, vbg_att_sup_loss 0.93857, vbg_phrcls_loss 1.24121, vbg_prc_auc 0.35281, 637.19 secs
(2) Validation stage ...
mimfg_phrcls_loss 1.19500, mimfg_prc_auc 0.67117, vbg_bbox_iou 0.11633, vbg_prc_auc 0.29602, 152.87 secs
New checkpoint saved: checkpoint_7_miss+miuc+vbss+vbou+vbuc+vbss=0.3740.pt
---- Epoch 8/200
(1) Training stage (lr = 0.000002) ...
l

loss 1.18210, mimfg_phrcls_loss 1.10140, mimfg_prc_auc 0.70142, vbg_vgbbox_loss 0.29072, vbg_att_sup_loss 0.65612, vbg_phrcls_loss 0.96152, vbg_prc_auc 0.40349, 650.76 secs
(2) Validation stage ...
mimfg_phrcls_loss 1.07890, mimfg_prc_auc 0.72944, vbg_bbox_iou 0.12229, vbg_prc_auc 0.35091, 158.11 secs
---- Epoch 26/200
(1) Training stage (lr = 0.000027) ...
loss 1.16224, mimfg_phrcls_loss 1.08993, mimfg_prc_auc 0.71431, vbg_vgbbox_loss 0.27292, vbg_att_sup_loss 0.61948, vbg_phrcls_loss 0.92056, vbg_prc_auc 0.42577, 647.56 secs
(2) Validation stage ...
mimfg_phrcls_loss 1.04895, mimfg_prc_auc 0.73967, vbg_bbox_iou 0.13392, vbg_prc_auc 0.35624, 154.08 secs
New checkpoint saved: checkpoint_26_miss+miuc+vbss+vbou+vbuc+vbss=0.4228.pt
---- Epoch 27/200
(1) Training stage (lr = 0.000009) ...
loss 1.13658, mimfg_phrcls_loss 1.06441, mimfg_prc_auc 0.72404, vbg_vgbbox_loss 0.26960, vbg_att_sup_loss 0.60310, vbg_phrcls_loss 0.91339, vbg_prc_auc 0.42411, 646.60 secs
(2) Validation stage ...
mimfg_

New checkpoint saved: checkpoint_44_miss+miuc+vbss+vbou+vbuc+vbss=0.4534.pt
---- Epoch 45/200
(1) Training stage (lr = 0.000080) ...
loss 1.05162, mimfg_phrcls_loss 0.99027, mimfg_prc_auc 0.75717, vbg_vgbbox_loss 0.24830, vbg_att_sup_loss 0.51800, vbg_phrcls_loss 0.83746, vbg_prc_auc 0.46911, 633.08 secs
(2) Validation stage ...
mimfg_phrcls_loss 0.93924, mimfg_prc_auc 0.77766, vbg_bbox_iou 0.16045, vbg_prc_auc 0.39060, 157.15 secs
New checkpoint saved: checkpoint_45_miss+miuc+vbss+vbou+vbuc+vbss=0.4551.pt
---- Epoch 46/200
(1) Training stage (lr = 0.000027) ...
loss 1.01932, mimfg_phrcls_loss 0.95610, mimfg_prc_auc 0.77116, vbg_vgbbox_loss 0.25224, vbg_att_sup_loss 0.51507, vbg_phrcls_loss 0.82102, vbg_prc_auc 0.50129, 641.11 secs
(2) Validation stage ...
mimfg_phrcls_loss 0.91573, mimfg_prc_auc 0.78056, vbg_bbox_iou 0.15389, vbg_prc_auc 0.39289, 159.05 secs
New checkpoint saved: checkpoint_46_miss+miuc+vbss+vbou+vbuc+vbss=0.4572.pt
---- Epoch 47/200
(1) Training stage (lr = 0.000009)

New checkpoint saved: checkpoint_64_miss+miuc+vbss+vbou+vbuc+vbss=0.4769.pt
---- Epoch 65/200
(1) Training stage (lr = 0.000080) ...
loss 0.96328, mimfg_phrcls_loss 0.90605, mimfg_prc_auc 0.78755, vbg_vgbbox_loss 0.23694, vbg_att_sup_loss 0.46133, vbg_phrcls_loss 0.78006, vbg_prc_auc 0.52018, 683.84 secs
(2) Validation stage ...
mimfg_phrcls_loss 0.87286, mimfg_prc_auc 0.80861, vbg_bbox_iou 0.16892, vbg_prc_auc 0.39262, 163.05 secs
---- Epoch 66/200
(1) Training stage (lr = 0.000027) ...
loss 0.92655, mimfg_phrcls_loss 0.86763, mimfg_prc_auc 0.80384, vbg_vgbbox_loss 0.22895, vbg_att_sup_loss 0.46466, vbg_phrcls_loss 0.76320, vbg_prc_auc 0.52892, 684.37 secs
(2) Validation stage ...
mimfg_phrcls_loss 0.85689, mimfg_prc_auc 0.81325, vbg_bbox_iou 0.16798, vbg_prc_auc 0.40998, 163.18 secs
New checkpoint saved: checkpoint_66_miss+miuc+vbss+vbou+vbuc+vbss=0.4770.pt
---- Epoch 67/200
(1) Training stage (lr = 0.000009) ...
loss 0.92304, mimfg_phrcls_loss 0.86995, mimfg_prc_auc 0.80455, vbg_vgb

mimfg_phrcls_loss 0.79860, mimfg_prc_auc 0.83422, vbg_bbox_iou 0.16448, vbg_prc_auc 0.42504, 164.51 secs
---- Epoch 85/200
(1) Training stage (lr = 0.000080) ...
loss 0.88695, mimfg_phrcls_loss 0.83322, mimfg_prc_auc 0.81492, vbg_vgbbox_loss 0.22611, vbg_att_sup_loss 0.42046, vbg_phrcls_loss 0.72395, vbg_prc_auc 0.57552, 662.98 secs
(2) Validation stage ...
mimfg_phrcls_loss 0.79977, mimfg_prc_auc 0.82704, vbg_bbox_iou 0.16158, vbg_prc_auc 0.40961, 165.57 secs
---- Epoch 86/200
(1) Training stage (lr = 0.000027) ...
loss 0.86947, mimfg_phrcls_loss 0.81652, mimfg_prc_auc 0.82354, vbg_vgbbox_loss 0.21831, vbg_att_sup_loss 0.40539, vbg_phrcls_loss 0.72231, vbg_prc_auc 0.58398, 666.94 secs
(2) Validation stage ...
mimfg_phrcls_loss 0.81089, mimfg_prc_auc 0.83518, vbg_bbox_iou 0.17111, vbg_prc_auc 0.42422, 165.25 secs
---- Epoch 87/200
(1) Training stage (lr = 0.000009) ...
loss 0.85315, mimfg_phrcls_loss 0.80351, mimfg_prc_auc 0.82811, vbg_vgbbox_loss 0.21430, vbg_att_sup_loss 0.38654, vbg

(2) Validation stage ...
mimfg_phrcls_loss 0.77464, mimfg_prc_auc 0.83999, vbg_bbox_iou 0.14863, vbg_prc_auc 0.42625, 154.19 secs
---- Epoch 106/200
(1) Training stage (lr = 0.000027) ...
loss 0.82342, mimfg_phrcls_loss 0.77200, mimfg_prc_auc 0.83753, vbg_vgbbox_loss 0.21047, vbg_att_sup_loss 0.37899, vbg_phrcls_loss 0.69673, vbg_prc_auc 0.59466, 641.99 secs
(2) Validation stage ...
mimfg_phrcls_loss 0.76604, mimfg_prc_auc 0.84847, vbg_bbox_iou 0.14994, vbg_prc_auc 0.43186, 151.71 secs
---- Epoch 107/200
(1) Training stage (lr = 0.000009) ...
loss 0.81663, mimfg_phrcls_loss 0.76903, mimfg_prc_auc 0.84123, vbg_vgbbox_loss 0.21195, vbg_att_sup_loss 0.36820, vbg_phrcls_loss 0.66493, vbg_prc_auc 0.61536, 639.63 secs
(2) Validation stage ...
mimfg_phrcls_loss 0.73554, mimfg_prc_auc 0.84932, vbg_bbox_iou 0.16004, vbg_prc_auc 0.43808, 151.80 secs
---- Epoch 108/200
(1) Training stage (lr = 0.000003) ...
loss 0.79611, mimfg_phrcls_loss 0.75172, mimfg_prc_auc 0.84538, vbg_vgbbox_loss 0.20559, v

---- Epoch 127/200
(1) Training stage (lr = 0.000009) ...
loss 0.77588, mimfg_phrcls_loss 0.73243, mimfg_prc_auc 0.85239, vbg_vgbbox_loss 0.20227, vbg_att_sup_loss 0.33580, vbg_phrcls_loss 0.62889, vbg_prc_auc 0.67726, 659.43 secs
(2) Validation stage ...
mimfg_phrcls_loss 0.73546, mimfg_prc_auc 0.85830, vbg_bbox_iou 0.14397, vbg_prc_auc 0.44332, 154.65 secs
---- Epoch 128/200
(1) Training stage (lr = 0.000003) ...
loss 0.77168, mimfg_phrcls_loss 0.72851, mimfg_prc_auc 0.85471, vbg_vgbbox_loss 0.20072, vbg_att_sup_loss 0.33841, vbg_phrcls_loss 0.62108, vbg_prc_auc 0.66269, 636.06 secs
(2) Validation stage ...
mimfg_phrcls_loss 0.72883, mimfg_prc_auc 0.85905, vbg_bbox_iou 0.14508, vbg_prc_auc 0.44437, 154.30 secs
---- Epoch 129/200
(1) Training stage (lr = 0.000001) ...
loss 0.77090, mimfg_phrcls_loss 0.72849, mimfg_prc_auc 0.85441, vbg_vgbbox_loss 0.20265, vbg_att_sup_loss 0.33066, vbg_phrcls_loss 0.61923, vbg_prc_auc 0.67081, 648.29 secs
(2) Validation stage ...
mimfg_phrcls_loss 0.71

In [49]:
!python ../train_phrase_grounding.py \
--pretrained_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/phrase_grounding/20250125_133402_mim-facts+vinbig_PhraseGrounder(aehrc-cxrmate-rrg24-uniformer,AdaptiveFiLM_MLP_BBoxRegression,128,256,256-128)" \
--epochs 100 \
--batches_per_epoch 400 \
--max_images_per_batch 10 \
--max_phrases_per_batch 1000 \
--max_phrases_per_image 45 \
--val_batch_size_factor 4.0 \
--raw_image_encoding "cxrmate-rrg24-uniformer-huggingface" \
--huggingface_model_name "aehrc/cxrmate-rrg24" \
--image_size 384 384 \
--image_local_feat_size 512 \
--num_regions 144 \
--regions_width 12 \
--regions_height 12 \
--img_aug_mode "random-color-and-spatial" \
--phrase_embedding_size 128 \
--phrase_grounding_mode "adaptive_film_based_pooling_mlp_with_bbox_regression" \
--visual_grounding_hidden_size 256 \
--phrase_mlp_hidden_dims 256 128 \
--num_train_workers 2 \
--num_val_workers 2 \
--gradient_accumulation_steps 4 \
--optimizer_name "adamw" \
--scheduler "exp-warmup+decay+cyclicdecay" \
--lr 1e-6 \
--warmup_decay_and_cyclic_decay_args "1e-6,3,8e-5,5,1e-6,8e-5,5,1e-6" \
--binary_multilabel_classif_loss_name "focal+bce+npbbce" \
--do_visual_grounding_with_bbox_regression \
--use_vinbig_for_train \
--use_vinbig_for_test \
--vinbig_phrase_embeddings_filepath \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/vinbig/label_phrase_embeddings(hash=561,2254045888337408829).pkl" \
--use_amp \
--save

script's arguments:
   epochs: 100
   batches_per_epoch: 400
   max_images_per_batch: 10
   max_phrases_per_batch: 1000
   max_phrases_per_image: 45
   val_batch_size_factor: 4.0
   checkpoint_folder: None
   pretrained_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/phrase_grounding/20250125_133402_mim-facts+vinbig_PhraseGrounder(aehrc-cxrmate-rrg24-uniformer,AdaptiveFiLM_MLP_BBoxRegression,128,256,256-128)
   pretrained_checkpoint_folder_paths: None
   freeze_image_encoder: False
   raw_image_encoding: cxrmate-rrg24-uniformer-huggingface
   huggingface_model_name: aehrc/cxrmate-rrg24
   num_regions: 144
   image_local_feat_size: 512
   image_encoder_pretrained_weights_path: None
   image_encoder_dropout_p: 0
   yolov8_model_name_or_path: None
   yolov8_model_alias: None
   phrase_embedding_size: 128
   regions_width: 12
   regions_height: 12
   qkv_size: None
   phrase_grounding_mode: adaptive_film_based_pooling_mlp_with_bbox_regression
   phrase_classi

self.phrase_grounding_masks.shape = (18000, 22, 144)
self.phrase_classification_labels.shape = (18000, 22)
Append additional labels to phrase_classification_labels
len(non_bbox_labels) = 6
non_bbox_labels = ['COPD', 'Lung tumor', 'Pneumonia', 'Tuberculosis', 'Other disease', 'No finding']
self.phrase_classification_labels.shape = (18000, 28)
Reorder phrases and phrase_embeddings
len(phrases) = 28
len(actual_phrases) = 28
phrase_embeddings.shape = (28, 128)
Generating train dataset and dataloader
len(train_indices) = 15000
No labels: 0
Edema: 13
Clavicle fracture: 27
Lung cyst: 33
COPD: 35
Lung cavity: 51
Emphysema: 81
Rib fracture: 90
Pneumothorax: 96
Enlarged PA: 131
Mediastinal shift: 150
Atelectasis: 187
Lung tumor: 291
Consolidation: 353
ILD: 397
Calcification: 458
Infiltration: 613
Tuberculosis: 750
Nodule/Mass: 841
Pneumonia: 917
Pleural effusion: 1038
Other lesion: 1154
Lung Opacity: 1331
Pulmonary fibrosis: 1621
Pleural thickening: 2010
Cardiomegaly: 2316
Aortic enlargement: 30

vbg_bbox_iou 0.20466, vbg_prc_auc 0.46147, 45.09 secs
New checkpoint saved: checkpoint_11_vbss+vbou+vbuc+vbss=0.3951.pt
---- Epoch 12/100
(1) Training stage (lr = 0.000009) ...
loss 1.02085, vbg_vgbbox_loss 0.19517, vbg_att_sup_loss 0.28247, vbg_phrcls_loss 0.54321, vbg_prc_auc 0.73536, 136.39 secs
(2) Validation stage ...
vbg_bbox_iou 0.20491, vbg_prc_auc 0.45984, 45.09 secs
---- Epoch 13/100
(1) Training stage (lr = 0.000003) ...
loss 1.00758, vbg_vgbbox_loss 0.19173, vbg_att_sup_loss 0.27795, vbg_phrcls_loss 0.53790, vbg_prc_auc 0.73757, 137.80 secs
(2) Validation stage ...
vbg_bbox_iou 0.20676, vbg_prc_auc 0.46065, 44.58 secs
New checkpoint saved: checkpoint_13_vbss+vbou+vbuc+vbss=0.3959.pt
---- Epoch 14/100
(1) Training stage (lr = 0.000001) ...
loss 1.02266, vbg_vgbbox_loss 0.19488, vbg_att_sup_loss 0.28373, vbg_phrcls_loss 0.54405, vbg_prc_auc 0.74010, 137.66 secs
(2) Validation stage ...
vbg_bbox_iou 0.20832, vbg_prc_auc 0.46164, 45.18 secs
New checkpoint saved: checkpoint_14_v

loss 0.87550, vbg_vgbbox_loss 0.18642, vbg_att_sup_loss 0.24010, vbg_phrcls_loss 0.44898, vbg_prc_auc 0.81360, 135.86 secs
(2) Validation stage ...
vbg_bbox_iou 0.20486, vbg_prc_auc 0.46827, 44.17 secs
---- Epoch 42/100
(1) Training stage (lr = 0.000009) ...
loss 0.85033, vbg_vgbbox_loss 0.18409, vbg_att_sup_loss 0.23292, vbg_phrcls_loss 0.43332, vbg_prc_auc 0.83230, 135.79 secs
(2) Validation stage ...
vbg_bbox_iou 0.20412, vbg_prc_auc 0.45879, 44.69 secs
---- Epoch 43/100
(1) Training stage (lr = 0.000003) ...
loss 0.83840, vbg_vgbbox_loss 0.18451, vbg_att_sup_loss 0.23239, vbg_phrcls_loss 0.42150, vbg_prc_auc 0.83370, 135.65 secs
(2) Validation stage ...
vbg_bbox_iou 0.20369, vbg_prc_auc 0.46189, 44.55 secs
---- Epoch 44/100
(1) Training stage (lr = 0.000001) ...
loss 0.83750, vbg_vgbbox_loss 0.18197, vbg_att_sup_loss 0.22769, vbg_phrcls_loss 0.42784, vbg_prc_auc 0.83310, 135.62 secs
(2) Validation stage ...
vbg_bbox_iou 0.20852, vbg_prc_auc 0.46083, 44.66 secs
---- Epoch 45/100
(1)